### Explore previous_application Dataset
https://www.kaggle.com/c/home-credit-default-risk

Ad-hoc exploration of `previous_application.csv` dataset to find numeric and categorical variables

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os
import shutil

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from preprocess import (do_data_cleaning, generate_encoders, add_onehot_col, add_label_col, fix_null_values)
from preprocess_bureau import (add_numeric_stats_cols, )

In [3]:
from IPython.display import display

In [4]:
df = pd.read_csv('./data/previous_application.csv')
print("shape",df.shape)
print("columns",df.columns)

shape (1670214, 37)
columns Index(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE',
       'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE',
       'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE',
       'CHANNEL_TYPE', 'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY',
       'CNT_PAYMENT', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
       'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL'],
      dtype='object')


In [5]:
df.dtypes

SK_ID_PREV                       int64
SK_ID_CURR                       int64
NAME_CONTRACT_TYPE              object
AMT_ANNUITY                    float64
AMT_APPLICATION                float64
AMT_CREDIT                     float64
AMT_DOWN_PAYMENT               float64
AMT_GOODS_PRICE                float64
WEEKDAY_APPR_PROCESS_START      object
HOUR_APPR_PROCESS_START          int64
FLAG_LAST_APPL_PER_CONTRACT     object
NFLAG_LAST_APPL_IN_DAY           int64
RATE_DOWN_PAYMENT              float64
RATE_INTEREST_PRIMARY          float64
RATE_INTEREST_PRIVILEGED       float64
NAME_CASH_LOAN_PURPOSE          object
NAME_CONTRACT_STATUS            object
DAYS_DECISION                    int64
NAME_PAYMENT_TYPE               object
CODE_REJECT_REASON              object
NAME_TYPE_SUITE                 object
NAME_CLIENT_TYPE                object
NAME_GOODS_CATEGORY             object
NAME_PORTFOLIO                  object
NAME_PRODUCT_TYPE               object
CHANNEL_TYPE             

In [6]:
df.select_dtypes('object').apply(pd.Series.nunique, axis=0).to_dict()

{'NAME_CONTRACT_TYPE': 4,
 'WEEKDAY_APPR_PROCESS_START': 7,
 'FLAG_LAST_APPL_PER_CONTRACT': 2,
 'NAME_CASH_LOAN_PURPOSE': 25,
 'NAME_CONTRACT_STATUS': 4,
 'NAME_PAYMENT_TYPE': 4,
 'CODE_REJECT_REASON': 9,
 'NAME_TYPE_SUITE': 7,
 'NAME_CLIENT_TYPE': 4,
 'NAME_GOODS_CATEGORY': 28,
 'NAME_PORTFOLIO': 5,
 'NAME_PRODUCT_TYPE': 3,
 'CHANNEL_TYPE': 8,
 'NAME_SELLER_INDUSTRY': 11,
 'NAME_YIELD_GROUP': 5,
 'PRODUCT_COMBINATION': 17}

In [7]:
label_cols = list(df.select_dtypes('object').apply(pd.Series.nunique, axis=0).to_dict().keys())
label_cols

['NAME_CONTRACT_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'FLAG_LAST_APPL_PER_CONTRACT',
 'NAME_CASH_LOAN_PURPOSE',
 'NAME_CONTRACT_STATUS',
 'NAME_PAYMENT_TYPE',
 'CODE_REJECT_REASON',
 'NAME_TYPE_SUITE',
 'NAME_CLIENT_TYPE',
 'NAME_GOODS_CATEGORY',
 'NAME_PORTFOLIO',
 'NAME_PRODUCT_TYPE',
 'CHANNEL_TYPE',
 'NAME_SELLER_INDUSTRY',
 'NAME_YIELD_GROUP',
 'PRODUCT_COMBINATION']

In [8]:
label_cols = ['NAME_CONTRACT_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'FLAG_LAST_APPL_PER_CONTRACT',
 'NAME_CASH_LOAN_PURPOSE',
 'NAME_CONTRACT_STATUS',
 'NAME_PAYMENT_TYPE',
 'CODE_REJECT_REASON',
 'NAME_TYPE_SUITE',
 'NAME_CLIENT_TYPE',
 'NAME_GOODS_CATEGORY',
 'NAME_PORTFOLIO',
 'NAME_PRODUCT_TYPE',
 'CHANNEL_TYPE',
 'NAME_SELLER_INDUSTRY',
 'NAME_YIELD_GROUP',
 'PRODUCT_COMBINATION',
 'NFLAG_INSURED_ON_APPROVAL']

df[label_cols].apply(pd.Series.nunique, axis=0)

NAME_CONTRACT_TYPE              4
WEEKDAY_APPR_PROCESS_START      7
FLAG_LAST_APPL_PER_CONTRACT     2
NAME_CASH_LOAN_PURPOSE         25
NAME_CONTRACT_STATUS            4
NAME_PAYMENT_TYPE               4
CODE_REJECT_REASON              9
NAME_TYPE_SUITE                 7
NAME_CLIENT_TYPE                4
NAME_GOODS_CATEGORY            28
NAME_PORTFOLIO                  5
NAME_PRODUCT_TYPE               3
CHANNEL_TYPE                    8
NAME_SELLER_INDUSTRY           11
NAME_YIELD_GROUP                5
PRODUCT_COMBINATION            17
NFLAG_INSURED_ON_APPROVAL       2
dtype: int64

In [9]:
# df.select_dtypes(['float64', 'int64']).agg(["min", "max", "mean", "median", "nunique"])
numerical_cols = list(df.select_dtypes(['float64', 'int64']).apply(pd.Series.nunique, axis=0).to_dict().keys())
print ("numerical_cols", numerical_cols)

numerical_cols ['SK_ID_PREV', 'SK_ID_CURR', 'AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE', 'HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']


In [10]:
df['RATE_DOWN_PAYMENT'].unique()

array([0.        ,        nan, 0.10397129, ..., 0.10340195, 0.04430024,
       0.10140051])

In [11]:
numerical_cols =['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 
                 'AMT_GOODS_PRICE', 
                 # 'HOUR_APPR_PROCESS_START', 
                 #'NFLAG_LAST_APPL_IN_DAY', 
                 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 
                 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 
                 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 
                 'DAYS_TERMINATION', 
                 # 'NFLAG_INSURED_ON_APPROVAL'
                ]

FEAT_CODE = "PREV_APP"
rename_cols = {}
for colname in numerical_cols:
    rename_cols[colname] = "{}_{}".format(FEAT_CODE, colname)

df[numerical_cols].agg(["min", "max", "mean", "median", "nunique"])

,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION
min,0.000000,0.000000e+00,0.000000e+00,-9.000000e-01,0.000000e+00,-0.000015,0.034781,0.373150,-2922.000000,-1.000000e+00,0.000000,-2922.000000,-2892.000000,-2801.000000,-2889.000000,-2874.000000
max,418058.145000,6.905160e+06,6.905160e+06,3.060045e+06,6.905160e+06,1.000000,1.000000,1.000000,-1.000000,4.000000e+06,84.000000,365243.000000,365243.000000,365243.000000,365243.000000,365243.000000
mean,15955.120659,1.752339e+05,1.961140e+05,6.697402e+03,2.278473e+05,0.079637,0.188357,0.773503,-880.679668,3.139511e+02,16.054082,342209.855039,13826.269337,33767.774054,76582.403064,81992.343838
median,11250.000000,7.104600e+04,8.054100e+04,1.638000e+03,1.123200e+05,0.051605,0.189122,0.835095,-581.000000,3.000000e+00,12.000000,365243.000000,-831.000000,-361.000000,-537.000000,-499.000000
nunique,357959.000000,9.388500e+04,8.680300e+04,2.927800e+04,9.388500e+04,207033.000000,148.000000,25.000000,2922.000000,2.097000e+03,49.000000,2838.000000,2892.000000,4605.000000,2873.000000,2830.000000


In [12]:
df["RATE_INTEREST_PRIVILEGED"].unique()

array([0.86733615,        nan, 0.83509514, 0.56871036, 0.84513742,
       0.852537  , 0.71564482, 0.63794926, 0.82082452, 0.4244186 ,
       0.83245243, 0.6448203 , 0.51374207, 0.54281184, 0.37315011,
       0.80655391, 0.50211416, 0.63742072, 0.72515856, 0.78065539,
       1.        , 0.54809725, 0.79069767, 0.48414376, 0.43657505,
       0.85465116])